## Number of Trials vs Coefficient Estimate Error

Tests the effect of Gaussian white noise on the estimated coefficients 

In [1]:
%load_ext autoreload
%autoreload 2

# Import Python packages
import pickle

# Package Imports
from sindy_bvp import SINDyBVP
from sindy_bvp.differentiators import PolyInterp
from sindy_bvp.library_builders import NoiseMaker

In [2]:
# Set file to load and stem for saving
load_stem = "./data/S3-P2-"
save_stem = "./data/Fig5b-S3-"

In [3]:
%%time

# Set a range of noise magnitudes to test
num_trials = [10, 20, 30, 40, 50,
              60, 70, 80, 90, 100,
              110, 120, 130, 140, 150,
              160, 170, 180, 190, 200]

# Since the data is noisy, we'll use a Polynomial Interpolation derivative method
poly = PolyInterp(diff_order=2, width=20, degree=5)

# Initialize NoiseMaker, which adds 1% noise then filters noisy signal
nm = NoiseMaker(noise_magnitude=0.01)

# Create an empty
results_list = []

print("Completed:", end=" ")
for trial_count in num_trials:
    # Initialize SINDyBVP object
    sbvp = SINDyBVP(file_stem = load_stem,
                    num_trials = trial_count,
                    differentiator = poly,
                    outcome_var = 'd^{2}u/dx^{2}',
                    noisemaker = nm,
                    known_vars = ['u', 'u^{2}', 'du/dx', 'f'],
                    dep_var_name = 'u',
                    ind_var_name = 'x')

    # Execute the optimization
    coeffs, plotter = sbvp.sindy_bvp()

    # Compute the S-L coeffs with Plotter analysis tool
    plotter.compute_sl_coeffs()
    
    # gather the learned coefficients and relevant metrics
    # And place into the results_list
    results_list.append({'num_trials': trial_count,
                         'loss': min(sbvp.groupreg.Losses),
                         'p': plotter.inferred_phi, 
                         'q': plotter.inferred_q,
                         'coeffs': coeffs})
    
    print(trial_count, end=" | ")

Completed: 10 | 20 | 30 | 40 | 50 | 60 | 70 | 80 | 90 | 100 | 110 | 120 | 130 | 140 | 150 | 160 | 170 | 180 | 190 | 200 | Wall time: 28min 6s


In [4]:
## Pickle the results
pickle.dump(results_list, open(save_stem+"results.pickle", "wb"))